# Calculating the gains from trading US stocks on Revolut

Stock is the quanty of stock changed in the trade:
* positive: stock is bought
* negative: stock is sold

Cash is the amount of USD changed in the trade:
* positive: stock is sold
* negative: stock is bought

Stock and cash have opposite sign in a given trade

In [ ]:
from utils import *

%load_ext autoreload
%autoreload 2

In [ ]:
logging.basicConfig(level=logging.INFO)

# List of trades

In [ ]:
df_trade = pd.concat(
    [
        get_df_trade("2020"),
        get_df_trade("2021"),
    ],
    axis = 0,
)
    
df_trade

In [ ]:
df1=df_trade[df_trade.ticker=="ZM"]
df1=df1[df1.action=="B"]
df1

In [ ]:
df1.sum()

In [ ]:
df1.dtypes

In [ ]:
df_trade.loc[0:20]

In [ ]:
df = df_trade.copy()
#df = df[df.ticker == "NCLH"]
# df = df[df.date < "2021-02"]
df

In [ ]:
for i in range(len(df)):
    row = df.iloc[i]
    if row["action"] == "B":
        continue
    # only rows with sales remain
    # now for a given sale, take if from the previous buys until there is nothing left
    stock_to_sell = abs(row["stock"])
    cash = row["cash"]
    ticker = row["ticker"]
    logging.debug(f"i={i}, stock_to_sell={stock_to_sell}")
    # now you do an action until there is nothing left, so a while loop
    nparray_stock_left = df["stock_left"].values
    nparray_price = df["price_with_fees"].values
    nparray_realised_gain = df["realised_gain"].values
    nparray_ticker = df["ticker"].values
    list_stock_sold_here = []
    list_price_sold_here = []
    list_cost_sold_here = []
    for j, stock_left in enumerate(nparray_stock_left):
        if nparray_ticker[j] != ticker:
            continue
        logging.debug(f"j={j}, stock_left={stock_left}")
        if np.isnan(stock_left):
            logging.debug(f"Skipping j={j} as NaN, as it is a buy")
        elif stock_left < 0:
            raise RuntimeError(f"stock_left is negative for j={j} stock_left={stock_left}")
        elif stock_left == 0:
            continue
        # we reach now cells with no zero values, all the cells with zero should be gone
        elif stock_to_sell <= stock_left:
            stock_sold_here = stock_to_sell
            list_stock_sold_here.append(stock_sold_here)
            price_sold_here = nparray_price[j]
            list_price_sold_here.append(price_sold_here)
            cost_sold_here = stock_sold_here * price_sold_here
            list_cost_sold_here.append(cost_sold_here)
            nparray_stock_left[j] = stock_left - stock_to_sell
            stock_to_sell = 0.0
            break
        else:
            # stock_to_sell is larger than the stock_left in this cell
            # so we sell this entire cell
            stock_sold_here = stock_left
            list_stock_sold_here.append(stock_sold_here)
            price_sold_here = nparray_price[j]
            list_price_sold_here.append(price_sold_here)
            cost_sold_here = stock_sold_here * price_sold_here
            list_cost_sold_here.append(cost_sold_here)
            nparray_stock_left[j] = 0 
            stock_to_sell = stock_to_sell - stock_left
    logging.debug(f"i={i}")
    logging.debug(f"nparray_stock_left={nparray_stock_left}")
    logging.debug(f"list_stock_sold_here={list_stock_sold_here}")
    logging.debug(f"list_price_sold_here={list_price_sold_here}")
    logging.debug(f"list_cost_sold_here={list_cost_sold_here}")
    total_cost = sum(list_cost_sold_here)
    logging.debug(f"total_cost={total_cost}")
    realised_gain = cash - total_cost
    logging.debug(f"realised_gain={realised_gain}")
    nparray_realised_gain[i] = realised_gain
    df["stock_left"] = nparray_stock_left
    df["realised_gain"] = nparray_realised_gain
df

In [ ]:
df2 = df.copy()
# df2 = df[df.ticker == "ZM"]
df2["stock_left_cost"] = df2["stock_left"] * df2["price_with_fees"]
df2["stock_price_variation"] = 100* (1 - df2["price"] / df2["price_with_fees"])
df2

In [ ]:
ticker = "SPCE"

In [ ]:
df2[df2.ticker==ticker].stock_left.sum()

In [ ]:
df2[df2.ticker==ticker].stock_left_cost.sum()

In [ ]:
df2[df2.ticker==ticker].stock_left_cost.sum()/df2[df2.ticker==ticker].stock_left.sum()

In [ ]:
0.12549843113773107/0.00050403

In [ ]:
0.13/0.00050403

In [ ]:
list(df2["stock"].values)

In [ ]:
df.loc[728]["stock"]

In [ ]:
df2[df2.ticker=="ZM"].stock_left.sum()

In [ ]:
208.000000*27.100000

In [ ]:
df2[df2.ticker=="ZM"].stock_left_cost.sum()

In [ ]:
df2[df2.ticker=="ZM"].realised_gain.sum()

In [ ]:
df[df.ticker=="NIO"].realised_gain.sum()

In [ ]:
df[df.ticker=="NIO"].stock_left.sum()

In [ ]:
df.realised_gain.sum()

In [ ]:
df.stock_left.sum()

# Sum for each ticker together

In [ ]:
df_sum = get_df_sum(df2)
df_sum

In [ ]:
df_sum[df_sum.stock_left > 0].sort_index()

In [ ]:
df_sum["realised_gain"].sum()

# Conclusion

A realised loss of 119.67 euro was obtained at Revolut in calendar year 2020.